In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
yello_trip_24_01 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [3]:
yello_trip_24_01['duration'] = yello_trip_24_01['tpep_dropoff_datetime'] - yello_trip_24_01['tpep_pickup_datetime']
yello_trip_24_01['duration'] = (yello_trip_24_01['duration'].dt.total_seconds()/60).round(2)
print(yello_trip_24_01.duration.describe(percentiles=[0.95, 0.99]))
yello_trip_24_01 = yello_trip_24_01.query("(duration > 1) and (duration < 60)")
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
yello_trip_24_01[categorical] = yello_trip_24_01[categorical].astype(str)

In [16]:
train_dicts = yello_trip_24_01[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
target = 'duration'
y_train = yello_trip_24_01[target].values


In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)

In [19]:
sns.histplot(y_pred, label='prediction')
sns.histplot(y_train, label='actual')
plt.show()

: 

In [ ]:
mean_squared_error(y_train, y_pred, squared=False)